In [6]:
#Libs:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import time

In [1]:
#extras: number of extra spaces at a line containing words i through j
#c: cost of an optimal arrangement of words
#p: each cut for a line is stored in a tab p
def dp(l,M):
    words = l.split()
    n = len(words)
    extras = [[0 for i in range(n)] 
                 for i in range(n)]
    lc = [[0 for i in range(n)] 
             for i in range(n)] 
    c = [0 for i in range(n+1)] 
    p = [0 for i in range(n+1)] 
    for i in range(n):           ###computing the extraspace
        if(len(words[i]) > M):
            return (False)
        else:
            extras[i][i]=M-len(words[i])
            for j in range(i+1,n):
                extras[i][j]=extras[i][j-1]-len(words[j])-1
                
    for i in range(n):  ###computing the linecost
        for j in range(i,n):
            if extras[i][j] < 0:
                lc[i][j] = 10 ** 20
            elif j == (n-1):
                lc[i][j] = 0
            else:
                lc[i][j] = extras[i][j] ** 3
    c[0] = 0
    
    p[0] = 0
     
    for j in range(1,n+1):      ####computing the minimal cost
        minimum = 10 ** 20
        lb = 0
        for i in range(1,j+1):
            cost = c[i-1] + lc[i-1][j-1]
            if cost <= minimum:
                minimum = cost
                lb = i
        c[j] = minimum
        p[j] = lb
         
        
    return [c,p]


def lines(j,line_breaks, result_set):  
    i = line_breaks[j]
    if i == 1:
        k = 1
    else:
        k = lines(i-1,line_breaks, result_set) + 1
        
    result_set.append(tuple((k,i,j)))
    #print(k,i,j)
    return k


#result_set: is the output of the lines
#input_text: array of words from text.split()
#This method returs the lines with words separed by one blank space
def printing_no_aligment(input_text, result_set):
    lines_filled_natual_blank_space = []
    line = ""
    
    for i in range(len(result_set)):
        #First word in each line
        first_word_index = result_set[i][1]
        
        #Last word in each line
        last_word_index = result_set[i][2]  
        
        for j in range(first_word_index-1, last_word_index):
            line +=input_text[j]+" "
        
        lines_filled_natual_blank_space.append(line.rstrip())
        line = ""
    
    return lines_filled_natual_blank_space


#To print the list/array formatted as string
def convert_words_to_full_text(s): 
    new = "" 
    for x in s: 
        new += x  
    return new 

#Assuming that "\n" is added to M so if M = 10, the final result will be 11.
#Used in LA
def adding_extra_blank_space_to_right(M, s):
    if(len(s)<M):
        count = 0;
        temp_str="";
        for i in range(0,(M-len(s))):
            temp_str+=' ';
        s=s+temp_str+"\n"
    else:
        s=s+"\n"
    
    return (s);

#Used in RA
def adding_extra_blank_space_to_left(M, s):
    if(len(s)<M):
        count = 0;
        temp_str = "";
       
        for i in range(0,(M-len(s))):
            temp_str+=' ';
        s= temp_str+s+"\n";
    else:
         s=s+"\n"
    return s;


#Used in CA
def adding_extra_blank_space_to_center(M, s):
    s = s.rstrip().lstrip();
    amount_of_blank = (M - len(s))/2;
    
    if(len(s)<M):
        count = 0;
        temp_str = "";
       
        for i in range(0,amount_of_blank):
            temp_str+=' ';
        
        s= temp_str+s+temp_str+"\n";
    else:
        s=s+"\n"
    return s;


def string_result(index, size, cost, approach, function, time, M):
    s = (index,size, cost,approach,function, str(time), str(M));
    return s;


def cost_funcion_based_on_total_number_of_additional_blank_space_LA(M,input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].rstrip(); #i+j
        total_additional_blank_space = (len(aligned[n]) - len(total_words_separated_by_one_blank_space));
         
        c+= (total_additional_blank_space);
    
    c= (c)**3;
    return c;

def cost_funcion_based_on_total_number_of_additional_blank_space_RA(M, input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].lstrip() #i+j
        total_additional_blank_space = len(aligned[n]) - len(total_words_separated_by_one_blank_space);
 
        c+= (total_additional_blank_space);
    
    c= (c)**3;
    return c;

def cost_funcion_based_on_total_number_of_additional_blank_space_CA(M,input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].replace("\n", "").lstrip().rstrip() #i+j
        total_additional_blank_space = len(aligned[n].replace("\n", "")) - len(total_words_separated_by_one_blank_space);
 
        c+= total_additional_blank_space;
    
    c= (c)**3;
    return c;


# Reading data

In [7]:
columns = ['Index', 'Size', 'Costs', 'Approach', 'Function', 'Time', 'M']

M = [10, 15, 20, 25, 30, 35, 40, 45, 50]


#input_data_(word_range_size)
input_data_1_4 = [['qqbz', 'b', 'rjs', 's', 'tlk', 'ooi', 'idod', 'il', 'qw', 'xbqx', 'j', 'b', 'ge', 'gobc', 'i'],
                 ['cm', 'fo', 'uz', 'vufo', 'z', 'gzuc', 'lyw', 'kxlc', 'gk', 'qp', 'nwg', 'tm', 'c', 'u', 'nl', 'ga', 'nes', 'q', 'fdij', 'w', 'wcw', 'pf', 'l', 'aj', 'o', 'cey', 'hg', 'thso', 'fcy', 'z', 'rw'],
                 ['wy', 'dpt', 'ptw', 's', 'pim', 'b', 'ifzb', 'van', 'gybr', 'ef', 'hvng', 'asz', 'uvl', 'fo', 'go', 'drkd', 'nkl', 'b', 'jdqp', 'sz', 'u', 'dgdv', 'jk', 'q', 'ql', 'fbk', 'a', 'fu', 'c', 'crx', 'lt', 'dsy', 'nkw', 'zykp', 's', 'q', 'zh', 'ih', 'no', 'r', 'cgsn', 'vtgg', 'omj', 'q'],
                 ['tqg', 'r', 'sig', 'xylu', 'kd', 'an', 'iqgo', 'aj', 'xh', 'ed', 'wv', 'rsmi', 'kpup', 'tt', 'jckd', 'p', 'tp', 'mfyq', 'vf', 't', 'hli', 'dahz', 's', 'kipw', 'fz', 'b', 'e', 'vun', 'mao', 'xfc', 'r', 'yn', 'y', 'tw', 'yx', 'avty', 'ti', 'wm', 'a', 'lk', 'scz', 'jy', 'f', 'zxzl', 'setj', 'h', 'bf', 'tpx', 'wxqi', 'qvav', 'wdz', 'hn', 'sof', 'ty', 'vwb', 'h', 'kc', 'kx', 'v'],
                 ['n', 'gsd', 'g', 'mrl', 'pdd', 'i', 'jcri', 'tljj', 'bz', 'gcne', 'trr', 'kh', 'k', 'lhwx', 'fkk', 'lri', 'w', 'm', 'v', 'zumn', 'hjwx', 'j', 'jq', 'tq', 'j', 'la', 'lxn', 'w', 'f', 'ta', 'wjn', 'l', 'aoiy', 'wkmz', 'omd', 'a', 'giiz', 'se', 'wmqa', 'p', 'ls', 'hk', 'kkzm', 'v', 'rw', 'sh', 'bnf', 'cd', 'c', 'hjt', 'so', 'tzzu', 'o', 'xi', 'slvv', 'j', 'hka', 'gdq', 'yx', 'o', 'g', 'ejrf', 'xiyu', 'am', 'p', 'at', 'l', 'us', 'nm', 'fo', 'v', 'p', 'vouw', 'ytk', 'k', 'kpgh'],
                 ['bwoj', 'r', 'yim', 'yk', 'alu', 'fzu', 'q', 'mrrb', 'fve', 'moyb', 'ssl', 'jm', 'xxcl', 'prhv', 'jzj', 'sq', 'zzdj', 'xhe', 'chpm', 't', 'b', 'hcf', 'gqfm', 'v', 'jdd', 'akh', 'l', 'vvde', 'r', 'dij', 'f', 'h', 'earj', 'mac', 'ghe', 'fj', 'zo', 'ibd', 'i', 'idfw', 'brfw', 'seqv', 'eq', 'kk', 'm', 'icqa', 'r', 'lf', 'xzyy', 'hxq', 'zwg', 'q', 'p', 'ergv', 'ngl', 'alqk', 'hd', 'y', 'px', 'imb', 'fwt', 'ty', 'crnr', 'f', 's', 'g', 'vhr', 'hs', 'm', 'qdib', 'bn', 'rpzg', 'xr', 'x', 'cyam', 'l', 'k', 'zkz', 'i', 'oev', 'uea', 'm', 'lcuc', 'c', 'szv', 'fl']]


input_data_1_5 =[['ur', 'twcov', 'ylvu', 'ci', 'gs', 'ezah', 'gjrt', 'vyphd', 'mswzg', 'lqz', 'wz', 'v'],
                ['kg', 'utbu', 'jeh', 'b', 'bgpom', 'qhvo', 'merbj', 'z', 'jq', 'rt', 'wn', 'md', 'rjtl', 'c', 'csx', 'kkviz', 'wyo', 'i', 'ornp', 'mrf', 'vs', 'yrwcb', 'kl', 'geyk', 'u', 'c', 'ag'],
                ['pka', 'cf', 'iuzpv', 'eucbd', 'qik', 'qs', 'yy', 'ngssc', 'ko', 'pjn', 'auz', 'cb', 'cu', 'tgp', 'ro', 'n', 'riv', 'oex', 'bmk', 'lsjlo', 'j', 'yvu', 'erqbh', 'oomsq', 'qhsdl', 'sq', 'giyp', 'qph', 'kimxh', 'vg', 'm', 'x', 'larma', 'sfa', 'wfooj', 'stn', 'jh'],
                ['spj', 'ce', 'cqazv', 'axfry', 'rgu', 'yysaj', 'ichan', 'pig', 'jdbm', 'w', 'g', 'wl', 'tf', 'y', 'rdyfy', 'pe', 'svob', 'hux', 'rxic', 'f', 'ctss', 'oc', 'aqe', 'umtt', 'c', 'l', 't', 't', 'af', 'bszx', 'x', 'th', 'itt', 'gfaik', 'axhwi', 'iy', 'yjmjy', 'm', 'nvgj', 'd', 'g', 'x', 'ky', 'mk', 'ftsg', 'vwdjf', 'jdcq', 'n', 'twcwk', 'g', 'c', 'gjb', 'nyj', 'p'],
                ['ck', 'iagn', 'rp', 'fs', 'njjt', 'wig', 'f', 'nclby', 'jf', 'iz', 'eb', 'sc', 'ye', 'dquct', 'frym', 'fzkr', 'ksf', 'iid', 'cttxg', 'lt', 'gathg', 'sbwx', 'rny', 'qhq', 'ite', 'm', 'bgsx', 'xmdiq', 'yxn', 'ciqqe', 'p', 'uqgda', 'trrd', 'ugwx', 'r', 'wz', 'fi', 'z', 'wwal', 'yxqz', 'oavdm', 'kt', 'od', 'y', 'tqcvb', 'n', 'fvlhz', 'i', 'fu', 'y', 'avkut', 'cxd', 'zoo', 'nxl', 'kw', 'lp', 'spzil', 'rg', 'ncs', 'ksqr', 'kgfan', 'sa', 'p'],
                ['okd', 'spxvr', 'l', 'cbg', 'nryc', 'dftfl', 'vpj', 's', 't', 'o', 'ayd', 'qzek', 'jhzv', 'xquck', 'g', 'to', 'x', 'araq', 'b', 'libs', 'h', 'fx', 'yvhih', 'oxk', 'cna', 'k', 'aud', 'gvscb', 'kiv', 'qge', 'qrzao', 'ch', 'beqgw', 'lh', 'f', 'u', 'uqxzw', 'kjk', 'fbt', 'sjae', 'qrw', 'ovs', 'rgvi', 'lmr', 'cm', 'lwl', 'u', 'caa', 'eg', 'f', 'sfhd', 'oica', 'oqb', 'i', 'zqn', 'udwx', 'smvr', 'm', 'rlg', 'eryj', 'sqqf', 'ur', 'hee', 'qbdx', 'lrnlk', 'uyo', 'qa', 'l', 'yyzgo', 'bfaxa', 'og', 'ml', 'b', 'na', 'pkc', 'ijr', 'tlg', 'w']]

input_data_1_6 = [['ncuk', 'y', 'nb', 'bbpny', 'fabis', 'uoppxp', 'knssmc', 'txnx', 'ej', 'c', 'naqg'],
                    ['clqwad', 'qmrki', 'hujmvi', 'h', 'ckm', 'gkihhi', 'iwaog', 'wqlplm', 'igao', 'ab', 'exhgfk', 'nus', 'geg', 'kqan', 'sc', 'pli', 'fg', 'xi', 'wn', 'vbqemr', 'x', 'x'],
                    ['tsic', 'hspaq', 'vpgz', 'c', 'pijx', 'y', 'fekmu', 'fjtg', 'oretzr', 'dcjw', 'ldvvuf', 'fg', 'sbsc', 'ngdy', 'smqeuy', 'jzj', 'btlx', 'iffo', 'vvsem', 'fqa', 'hfr', 'bwm', 'xtekp', 'zrmcz', 'pq', 'ng', 's', 'j', 'tbocdu', 'ada', 'm', 'kkka', 'b', 't'],
                    ['zffumo', 'l', 'pwfumh', 'jhxkv', 'bkqdvp', 'efvd', 'lxxxbx', 'yzdrs', 'drzk', 'gnzrr', 'drbet', 'v', 'sk', 'g', 'kxbvvk', 'qgnni', 'cov', 'e', 'qykqle', 'w', 'ej', 'fjmosf', 'ub', 'y', 'jxv', 'wtrm', 'jlorsn', 'r', 'yojal', 'c', 'rwgu', 'x', 'mo', 'aofqm', 'xijj', 'mp', 't', 'xvg', 'tyyd', 'khr', 'd', 'qabybr', 'cgtner', 'kyplb'],
                    ['n', 'be', 'eexq', 'pmd', 'k', 'a', 'zg', 'zhu', 'h', 'eb', 'gsdizk', 'hnype', 'h', 'wkxyp', 'kiwd', 'pwq', 'lbku', 'tsyksx', 'ls', 'okgtw', 'lmiwz', 'p', 'acpnlf', 'gudw', 'sk', 'f', 'khproy', 'udka', 'm', 'q', 'pt', 'qt', 'zlub', 'of', 'hkxm', 'xt', 'dbz', 'bklc', 'qtcos', 'p', 'gi', 'cpme', 'grddaf', 'nl', 'pr', 'w', 'cqt', 'h', 'tlaibo', 'iebw', 'dkrgug', 'ed', 'dbfjv', 'zfa', 'glxbk', 'crhm', 'x', 'qup', 'zefhg', 'wmaxc'],
                    ['b', 'fwlut', 'yr', 'so', 'phcgc', 'myf', 'dgishz', 'bo', 'leshyy', 'gzmtk', 'ksms', 'dicn', 'eibn', 'q', 'oc', 'j', 'o', 'anqcau', 'bc', 'a', 'wqpx', 'mwfiay', 'x', 'cr', 'e', 'vyqmw', 'jqojee', 'zvwuqa', 'zduzie', 'djser', 'yvi', 'tf', 'ffzj', 'pu', 'kjs', 'maf', 'yazct', 'ndxxxn', 'izuus', 'mdp', 'ihtfsy', 'torgv', 'cak', 'gdc', 'qbbqqm', 'awl', 'qxuzd', 'iez', 'keen', 'y', 'i', 'op', 'mfew', 'vyhwrd', 'jnback', 'eomnvg', 'sbtn', 'uxtn', 'gpc', 'pc', 'irtp', 'jzxw', 'hib', 'plu', 'vl', 'j']]


input_data_1_7 = [['pyn', 'nf', 'bcjfop', 'qxeiyy', 'ek', 'ctdtyob', 'efgumr', 'rdoznsm', 'ddo'],
                    ['ksjc', 'kvpwoa', 'hi', 'ezyo', 'er', 'qlgls', 'hvxmws', 'lfgupv', 'karzrrx', 'pfs', 'qdehsyt', 'neiul', 'sq', 'i', 'ibqk', 'lhwel', 'bcscs', 'js', 'k', 'cq', 'q'],
                    ['w', 'nl', 'grwn', 'ybtemk', 'kyxt', 'vum', 'bpo', 'wi', 'azilxpy', 'q', 'guqpj', 'pmgq', 'edecd', 'uf', 'ggtw', 'tszb', 'yurohdr', 'is', 'yjky', 'n', 'rhpfcpb', 'dyuo', 'd', 'pxd', 'bjny', 'dldzhul', 'eax', 'abbw', 'ugdikjb', 'nme', 'pnomie'],
                    ['qk', 'i', 'k', 'ss', 'ake', 'bz', 'gbi', 'av', 'vbfsf', 'mjnxyk', 'dgpgjmf', 'gdiedoc', 'tzkfkx', 'brvkzbz', 'hbwnv', 'opwqcnv', 'ipipbea', 'iyurfcs', 'igumihc', 's', 'zbd', 'y', 'wprxhf', 'ybmbu', 'xpdui', 'dw', 'y', 'wwi', 'xmejvn', 'mv', 'wfjteci', 'gdqezli', 'ddbdfo', 'mcawsk', 'c', 'h', 'tpxj', 'omrtmtm', 'e'],
                    ['qastjgj', 'o', 'kdx', 'wyrop', 'lhh', 'kjgjo', 'ijhzr', 'o', 'mgtuwuq', 'kcpwy', 'rvoydbf', 'pgqplnp', 'ivg', 'lzlcgx', 'yfs', 'aqek', 'flr', 'kwyqy', 'fkios', 'ag', 'uqxj', 'x', 'y', 'velf', 'vi', 'idsne', 'za', 'csdit', 'trwtnfj', 'h', 'bkmnqj', 'w', 'ik', 'wopl', 'haav', 'r', 'iebsc', 'ngex', 'f', 'biswm', 'ufj', 'rrl', 'pshseh', 'uzaz', 'wxjc', 'hnfh', 'etxvh', 'jpxvkjm', 'u', 'uig', 'ednj', 'dga'],
                    ['fbabq', 'umgl', 'jihtiwi', 'xxehknv', 'e', 'yn', 'lwovf', 'tzcgpt', 'xyqwosj', 'pycpziv', 'nkwmfac', 's', 'sjrwtrx', 'jte', 'bjuvzfh', 'hy', 'stct', 'dtgeat', 'zqqb', 'b', 'eweyyxy', 'fmhdkcr', 'xu', 'eooq', 'oknyz', 'uca', 'i', 'niugli', 'm', 'va', 'in', 'hiz', 'r', 'bkahv', 'ras', 'z', 'm', 'xqajvlj', 'n', 'rducw', 'wnejrnj', 'snbdy', 'zq', 'e', 'gkgghpk', 'vtl', 'm', 'vyjva', 'uswu', 'zac', 'tku', 'rliy', 'ocyprvg', 'f', 'zpz', 'icjmwz', 'l', 'ypues', 'ayderkn', 'iy', 'a', 'x']]


input_data_1_8 = [['pg', 'ascowy', 'jis', 'wuvkjqif', 'odwp', 'vnvwwho', 'onp', 'hlijecxt', 'q'],
                ['fuacmv', 'rdwimqha', 'lufyyar', 'iucjtrg', 'vnuzc', 'isdnooy', 'yyidvc', 'yzhiaccq', 'h', 'sklikmg', 'mj', 'fmaqld', 'ydo', 'vauewx', 'n', 'st', 'bs'],
                ['npd', 'zwu', 'zataaxhl', 'nap', 'o', 'vr', 'curo', 'mfpycj', 'owoozeeg', 'xd', 'j', 'yvw', 'upwxbxy', 't', 'julak', 'ojocd', 'l', 'rtxl', 'espni', 'fhewdjsu', 'xgw', 'zhvowi', 'htsa', 'fbp', 'lqnpm', 'aup', 'dgizg', 'txu', 'yshwq', 'yntj'],
                ['zfbjkbsc', 'xalmixgy', 'eb', 'si', 'lq', 'nrbtysq', 'sitx', 'v', 'b', 'wpoboo', 'pl', 'tyumqgk', 'ejwus', 'plrkesgd', 'sexladwr', 'sr', 'a', 'cju', 'rmjflg', 'hfpqism', 'enywahyd', 'wqhxv', 'mrkxyue', 'imcr', 'ems', 'rkvyuta', 'ojrlc', 'qzotuuj', 'sjsn', 'xexdo', 'nqqpfgkb', 'pcasy', 'btd', 'ovjb', 't'],
                ['nrmx', 'ltue', 'uqarwaqw', 's', 'lurfasxa', 'wsp', 'k', 'byuqxw', 'mptp', 'ufcy', 'qs', 'mt', 'kqlcn', 'qxghzy', 'a', 'xwx', 'hqvu', 'lj', 'bbxpukwg', 'rxqoky', 'lvvqiqzf', 'cbnqlpw', 'hszpcgo', 'pcijlb', 'iy', 'x', 'mhuqketq', 'nqndyxhg', 'chsb', 'qutsy', 'baxt', 'tmx', 'nqa', 'xsfvhou', 'vday', 'e', 'g', 'fczewi', 'bbn', 'gxevasi', 'rvsfove', 'qlnvbyy', 'wnl', 'azwrt', 'tviqu', 'c'],
                ['vietxl', 'wcacbtvt', 'rq', 'vmjz', 'hiyw', 'iuxqrnr', 'aqmxcpij', 'rtr', 'w', 'mtttdj', 'j', 'ybwplbgh', 'wtwuctje', 'qqa', 'ihcd', 'qgdfb', 'fvfvsqey', 'go', 'pgorbf', 'zil', 'bzhk', 'dghym', 'i', 'qn', 'naoyntv', 'hzorwjqu', 'xlysxg', 'emxuvfa', 'l', 'rm', 'u', 'f', 'vs', 'qmsv', 'lfylpys', 'ttkt', 'kjab', 'pzfomp', 'huqut', 'nhfoi', 'bxffackl', 'vgvzgubn', 'hdwxi', 'i', 'yvz', 'iduig', 'urc', 'qmo', 'xrrz', 'nhrigfmw', 'bk', 'qt', 'n', 'yx', 'xq', 'ugdnzwp', 'u']]


### 1. Dynamic Programming Left Alignment

In [10]:
def running_dynamic_LA_over_input_data_and_Ms(M, input_data, columns):    
    results_DPLA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= dp(" ".join(input_data[i]), M[z])
            
            #End processing time: 
            end_time = time.time() - start_time

            if (result):
                # outputs: (k,i,j) indicating that words i, . . . , j are printed on line k.
                result_set = []
                words = (" ".join(input_data[i])).split()

                lines(len(words), result[1], result_set)

                ##Get list of lines with words with one blank space separating each other
                t = printing_no_aligment(words, result_set)

                alignedLA = []
                for j in range(len(t)):
                    alignedLA.append(adding_extra_blank_space_to_right(M[z], t[j]))

                total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_LA(M[z], input_data[i], alignedLA)    
                results_DPLA.append(string_result(index, len(' '.join(input_data[i])) , total_cost , "Dynamic Programming", "LA", end_time, M[z]));    
                index+=1; 
            else:
                print("There is one or more words > M in your text")
    
    results_DPLA = pd.DataFrame(results_DPLA , index=None, columns=columns)
    
    return results_DPLA


In [11]:
DPLA_result_1_4 = running_dynamic_LA_over_input_data_and_Ms(M, input_data_1_4, columns)
DPLA_result_1_5 = running_dynamic_LA_over_input_data_and_Ms(M, input_data_1_5, columns)
DPLA_result_1_6 = running_dynamic_LA_over_input_data_and_Ms(M, input_data_1_6, columns)
DPLA_result_1_7 = running_dynamic_LA_over_input_data_and_Ms(M, input_data_1_7, columns)
DPLA_result_1_8 = running_dynamic_LA_over_input_data_and_Ms(M, input_data_1_8, columns)

In [13]:
#Generate CSVs
DPLA_result_1_4.to_csv('resultsDPLA_1_4.csv', sep=',' , header=None, index=None)
DPLA_result_1_5.to_csv('resultsDPLA_1_5.csv', sep=',' , header=None, index=None)
DPLA_result_1_6.to_csv('resultsDPLA_1_6.csv', sep=',' , header=None, index=None)
DPLA_result_1_7.to_csv('resultsDPLA_1_7.csv', sep=',' , header=None, index=None)
DPLA_result_1_8.to_csv('resultsDPLA_1_8.csv', sep=',' , header=None, index=None)

### 2. Dynamic Programming Right Alignment

In [14]:
def running_dynamic_RA_over_input_data_and_Ms(M, input_data, columns):    
    results_DPRA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= dp(" ".join(input_data[i]), M[z])
            
            #End processing time: 
            end_time = time.time() - start_time


            if (result):
                # outputs: (k,i,j) indicating that words i, . . . , j are printed on line k.
                result_set = []
                words = (" ".join(input_data[i])).split()

                lines(len(words), result[1], result_set)

                ##Get list of lines with words with one blank space separating each other
                t = printing_no_aligment(words, result_set)

                alignedRA = []
                for j in range(len(t)):
                    alignedRA.append(adding_extra_blank_space_to_left(M[z], t[j]))

                total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_RA(M[z], input_data[i], alignedRA)    
                results_DPRA.append(string_result(index, len(' '.join(input_data[i])) , total_cost , "Dynamic Programming", "RA", end_time, M[z]));    
                index+=1;
                #print(convert_words_to_full_text(alignedRA))
            else:
                print("There is one or more words > M in your text")
                
    results_DPRA = pd.DataFrame(results_DPRA , index=None, columns=columns)
    
    return results_DPRA;
        

In [15]:
DPRA_result_1_4 = running_dynamic_RA_over_input_data_and_Ms(M, input_data_1_4, columns)
DPRA_result_1_5 = running_dynamic_RA_over_input_data_and_Ms(M, input_data_1_5, columns)
DPRA_result_1_6 = running_dynamic_RA_over_input_data_and_Ms(M, input_data_1_6, columns)
DPRA_result_1_7 = running_dynamic_RA_over_input_data_and_Ms(M, input_data_1_7, columns)
DPRA_result_1_8 = running_dynamic_RA_over_input_data_and_Ms(M, input_data_1_8, columns)

In [16]:
#Generate CSVs
DPRA_result_1_4.to_csv('resultsDPRA_1_4.csv', sep=',' , header=None, index=None)
DPRA_result_1_5.to_csv('resultsDPRA_1_5.csv', sep=',' , header=None, index=None)
DPRA_result_1_6.to_csv('resultsDPRA_1_6.csv', sep=',' , header=None, index=None)
DPRA_result_1_7.to_csv('resultsDPRA_1_7.csv', sep=',' , header=None, index=None)
DPRA_result_1_8.to_csv('resultsDPRA_1_8.csv', sep=',' , header=None, index=None)

### 3. Dynamic Programming Center Alignment

In [20]:
def running_dynamic_CA_over_input_data_and_Ms(M, input_data, columns):    
    results_DPCA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= dp(" ".join(input_data[i]), M[z])
            
            #End processing time: 
            end_time = time.time() - start_time

            if (result):
                # outputs: (k,i,j) indicating that words i, . . . , j are printed on line k.
                result_set = []
                words = (" ".join(input_data[i])).split()

                lines(len(words), result[1], result_set)

                ##Get list of lines with words with one blank space separating each other
                t = printing_no_aligment(words, result_set)

                alignedCA = []
                for j in range(len(t)):
                    alignedCA.append(adding_extra_blank_space_to_center(M[z], t[j]))

                total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_CA(M[z], input_data[i], alignedCA)    
                results_DPCA.append(string_result(index, len(' '.join(input_data[i])), total_cost , "Dynamic Programming", "CA", end_time, M[z]));    
                index+=1;
                #print(convert_words_to_full_text(alignedCA))
            else:
                print("There is one or more words > M in your text")
                
    results_DPCA = pd.DataFrame(results_DPCA , index=None, columns=columns)
    return results_DPCA

In [21]:
DPCA_result_1_4 = running_dynamic_CA_over_input_data_and_Ms(M, input_data_1_4, columns)
DPCA_result_1_5 = running_dynamic_CA_over_input_data_and_Ms(M, input_data_1_5, columns)
DPCA_result_1_6 = running_dynamic_CA_over_input_data_and_Ms(M, input_data_1_6, columns)
DPCA_result_1_7 = running_dynamic_CA_over_input_data_and_Ms(M, input_data_1_7, columns)
DPCA_result_1_8 = running_dynamic_CA_over_input_data_and_Ms(M, input_data_1_8, columns)

In [22]:
#Generate CSVs
DPCA_result_1_4.to_csv('resultsDPCA_1_4.csv', sep=',' , header=None, index=None)
DPCA_result_1_5.to_csv('resultsDPCA_1_5.csv', sep=',' , header=None, index=None)
DPCA_result_1_6.to_csv('resultsDPCA_1_6.csv', sep=',' , header=None, index=None)
DPCA_result_1_7.to_csv('resultsDPCA_1_7.csv', sep=',' , header=None, index=None)
DPCA_result_1_8.to_csv('resultsDPCA_1_8.csv', sep=',' , header=None, index=None)